<a id='spis'></a>

<h1>Spis treści<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#helpers" data-toc-modified-id="helpers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>helpers</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#funkcja-pusty-harmonogram" data-toc-modified-id="funkcja-pusty-harmonogram-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>funkcja pusty harmonogram</a></span></li><li><span><a href="#funkcja-konwersja-planu-spłat-na-harmonogram" data-toc-modified-id="funkcja-konwersja-planu-spłat-na-harmonogram-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>funkcja konwersja planu spłat na harmonogram</a></span></li><li><span><a href="#funkcja-łączy-i-agreguje-harmonogramy" data-toc-modified-id="funkcja-łączy-i-agreguje-harmonogramy-2.0.3"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>funkcja łączy i agreguje harmonogramy</a></span></li><li><span><a href="#funkcja-modyfikacja-harmonogramu" data-toc-modified-id="funkcja-modyfikacja-harmonogramu-2.0.4"><span class="toc-item-num">2.0.4&nbsp;&nbsp;</span>funkcja modyfikacja harmonogramu</a></span></li><li><span><a href="#funkcja-przesuwanie-dat-płatności" data-toc-modified-id="funkcja-przesuwanie-dat-płatności-2.0.5"><span class="toc-item-num">2.0.5&nbsp;&nbsp;</span>funkcja przesuwanie dat płatności</a></span></li><li><span><a href="#funkcja-usuwanie-płatności-z-datą-sprzed-daty-danych" data-toc-modified-id="funkcja-usuwanie-płatności-z-datą-sprzed-daty-danych-2.0.6"><span class="toc-item-num">2.0.6&nbsp;&nbsp;</span>funkcja usuwanie płatności z datą sprzed daty danych</a></span></li></ul></li></ul></li><li><span><a href="#Przykład-1---pojedynczy-plan-spłaty" data-toc-modified-id="Przykład-1---pojedynczy-plan-spłaty-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Przykład 1 - pojedynczy plan spłaty</a></span></li><li><span><a href="#Przykład-2---Harmonogram-składany-z-wielu-planów-spłaty" data-toc-modified-id="Przykład-2---Harmonogram-składany-z-wielu-planów-spłaty-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Przykład 2 - Harmonogram składany z wielu planów spłaty</a></span></li><li><span><a href="#Przykład-3---modyfikowanie-płatności-w-zakresie-dat" data-toc-modified-id="Przykład-3---modyfikowanie-płatności-w-zakresie-dat-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Przykład 3 - modyfikowanie płatności w zakresie dat</a></span></li><li><span><a href="#Przykład-4---przesuwanie-płatności-w-harmonogramie" data-toc-modified-id="Przykład-4---przesuwanie-płatności-w-harmonogramie-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Przykład 4 - przesuwanie płatności w harmonogramie</a></span></li><li><span><a href="#Przykład-5---usuwanie-płatności-sprzed-daty-analizy" data-toc-modified-id="Przykład-5---usuwanie-płatności-sprzed-daty-analizy-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Przykład 5 - usuwanie płatności sprzed daty analizy</a></span></li></ul></div>

### imports

In [1]:
import numpy as np
import pandas as pd
import calendar as cl
from datetime import datetime

### helpers

##### funkcja pusty harmonogram

[<Spis treści](#spis)

In [2]:
def sched_empty():
    df = pd.DataFrame(columns=['data_platnosci', 'proba', 'kapital','odsetki'])
    return df

##### funkcja konwersja planu spłat na harmonogram

[<Spis treści](#spis)

In [3]:
def sched_payment_plan(freq_cde, payments_cnt, first_payment_dte,  proba_pct, capital_amt, interest_amt):
    df=sched_empty()  
    
    try:
        if freq_cde in ['M','Q','Y']:
            #ustalennie mnożnika miesięcy dla innych niż M częstotliwości
            freq_mltp=1
            if freq_cde=='Q':
                freq_mltp=3
            elif freq_cde=='Y':
                freq_mltp=12
        else:
            raise ValueError
    except ValueError:
        print('Niewłaściwa częstotliwość płatności:', freq_cde) 
        return df
    
    try: 
        # data z string na date
        first_payment_dte=datetime.strptime(first_payment_dte, '%Y-%m-%d')
    except ValueError:
        print('Niewłaściwa data:', first_payment_dte)
        return df
    
    try: 
        if payments_cnt<=0:
            raise ValueError
    except ValueError:
        print('Niewłaściwa ilość płatności:', payments_cnt)
        return df
    
    try: 
        if proba_pct<0 or proba_pct>1 or capital_amt<0 or interest_amt<0 :
            raise ValueError
    except ValueError:
        print('Niewłaściwe parametry spłaty:', proba_pct,capital_amt, interest_amt)
        return df    
    
    first_payment_year = first_payment_dte.year
    first_payment_month = first_payment_dte.month
    first_payment_day=first_payment_dte.day
    first_payment_id=first_payment_year*12+first_payment_month #licznik miesiecy 'naszej ery'
    
    next_payment_dte=first_payment_dte
    
    for i in range(payments_cnt):
        df.loc[i,df.columns[0]]=next_payment_dte
        df.loc[i,df.columns[1]]=proba_pct
        df.loc[i,df.columns[2]]=capital_amt
        df.loc[i,df.columns[3]]=interest_amt
        
        #wyliczenie dnia następnej płatności przesuwają licznik miesięcy naszej ery i wyznacząc dzień miesiąca, dla krótkich miesiący
        next_payment_id=first_payment_id+(i+1)*freq_mltp
        next_payment_year=next_payment_id//12 + (-1 if next_payment_id%12==0 else 0)
        next_payment_month=next_payment_id%12 + (0 if next_payment_id%12!=0 else 12)
        
        next_payment_days_no = cl.monthrange(next_payment_year, next_payment_month)[1] #liczba dni w miesiącu daty następnej płatności
        next_payment_day=first_payment_day if first_payment_day<=next_payment_days_no else next_payment_days_no
        next_payment_dte= datetime(next_payment_year, next_payment_month, next_payment_day)
    return df

##### funkcja łączy i agreguje harmonogramy

[<Spis treści](#spis)

In [4]:
def sched_concat(sp_list):
    col=list(sp_list[0].columns[0:2])
    df=(pd.concat(sp_list).groupby(col).sum()
        .sort_values(by=col)
        .reset_index())   
    return df

##### funkcja modyfikacja harmonogramu

[<Spis treści](#spis)

In [5]:
 def sched_modify(sched, 
                  start_dte, 
                  end_dte, 
                  chng_proba_ind=0, 
                  chng_cap_ind=0, 
                  chng_int_ind=0, 
                  new_proba=0, 
                  new_cap=0, 
                  new_int=0):
        
    df=sched_empty()
    
    try: 
        # data z string na date
        start_dte=datetime.strptime(start_dte, '%Y-%m-%d')
        end_dte=datetime.strptime(end_dte, '%Y-%m-%d')       
    except ValueError:
        print('Niewłaściwa data:', start_dte,end_dte)
        return df
    try: 
        if end_dte<start_dte:
            raise ValueError
    except ValueError:
        print('Start_dte późniejsze niż end_dte:', start_dte,end_dte)
        return df
    try: 
        if start_dte>sched.iloc[:,0].max():
            raise ValueError
    except ValueError:
        print('Start_dte późniejsze od max data raty')
        return df
 
    try: 
        if end_dte<sched.iloc[:,0].min():
            raise ValueError
    except ValueError:
        print('End_dte wcześniejsze niż min data raty')
        return df

    #część harmonogramu bez zmian
    sched_static=sched.loc[(sched.iloc[:,0]<start_dte) | (sched.iloc[:,0]>end_dte),: ].copy()
    #część harmonogramu do zmiany
    sched_to_chng=sched.loc[(sched.iloc[:,0]>=start_dte) & (sched.iloc[:,0]<=end_dte),: ].copy()
    
    if chng_proba_ind:
        sched_to_chng.iloc[:,1]=new_proba
    if chng_cap_ind:
        sched_to_chng.iloc[:,2]=new_cap
    if chng_int_ind:
        sched_to_chng.iloc[:,3]=new_int        
    
    df=sched_concat([sched_static,sched_to_chng])
    
    return df

##### funkcja przesuwanie dat płatności

[<Spis treści](#spis)

In [6]:
 def sched_shift(sched,start_dte,end_dte,shift_months_cnt):
        
    df=sched_empty()
    
    try: 
        # data z string na date
        start_dte=datetime.strptime(start_dte, '%Y-%m-%d')
        end_dte=datetime.strptime(end_dte, '%Y-%m-%d')       
    except ValueError:
        print('Niewłaściwa data:', start_dte,end_dte)
        return df
    try: 
        if end_dte<start_dte:
            raise ValueError
    except ValueError:
        print('Start_dte późniejsze niż end_dte:', start_dte,end_dte)
        return df
    try: 
        if start_dte>sched.iloc[:,0].max():
            raise ValueError
    except ValueError:
        print('Start_dte późniejsze od max data raty')
        return df
 
    try: 
        if end_dte<sched.iloc[:,0].min():
            raise ValueError
    except ValueError:
        print('End_dte wcześniejsze niż min data raty')
        return df

    #część harmonogramu bez zmian
    sched_static=sched.loc[(sched.iloc[:,0]<start_dte) | (sched.iloc[:,0]>end_dte),: ].copy()
    #część harmonogramu do zmiany
    sched_to_chng=sched.loc[(sched.iloc[:,0]>=start_dte) & (sched.iloc[:,0]<=end_dte),: ].copy()
    
    if shift_months_cnt!=0:
        for index, row in sched_to_chng.iterrows():
            new_payment_id=row[0].year*12+row[0].month+shift_months_cnt
            new_payment_year=new_payment_id//12 + (-1 if new_payment_id%12==0 else 0)
            new_payment_month=new_payment_id%12 + (0 if new_payment_id%12!=0 else 12)
            
            current_payment_day=row[0].day
            new_payment_days_no = cl.monthrange(new_payment_year, new_payment_month)[1] #liczba dni w miesiącu daty następnej płatności
            new_payment_day=current_payment_day if current_payment_day<=new_payment_days_no else new_payment_days_no
            new_payment_dte= datetime(new_payment_year, new_payment_month, new_payment_day)
            sched_to_chng.loc[index,sched_to_chng.columns[0]]=new_payment_dte
            
    df=sched_concat([sched_static,sched_to_chng])
    #display(sched_to_chng)
    return df

##### funkcja usuwanie płatności z datą sprzed daty danych

[<Spis treści](#spis)

In [7]:
def sched_dates_valid(sched, data_dte):
    df=sched_empty()
    
    try: 
        # data z string na date
        data_dte=datetime.strptime(data_dte, '%Y-%m-%d')
    except ValueError:
        print('Niewłaściwa data danych:', data_dte)
        return df

    if data_dte>sched.iloc[:,0].max(): #data danych późniejsza od dat harmonogramu
        return df
    elif data_dte<sched.iloc[:,0].min(): #data danych sprzed płatności z harmonogramu
        return sched
    else:
        #część harmonogramu bez zmian, tj. po dacie danych
        sched_valid=sched.loc[(sched.iloc[:,0]>=data_dte),: ].copy()
        return sched_valid

### Przykład 1 - pojedynczy plan spłaty

[<Spis treści](#spis)

In [8]:
sched_payment_plan(freq_cde='M', payments_cnt=15, first_payment_dte='2022-12-30',  proba_pct=0.9, capital_amt=100, interest_amt=10)

,data_platnosci,proba,kapital,odsetki
0,2022-12-30 00:00:00,0.9,100,10
1,2023-01-30 00:00:00,0.9,100,10
2,2023-02-28 00:00:00,0.9,100,10
3,2023-03-30 00:00:00,0.9,100,10
4,2023-04-30 00:00:00,0.9,100,10
5,2023-05-30 00:00:00,0.9,100,10
6,2023-06-30 00:00:00,0.9,100,10
7,2023-07-30 00:00:00,0.9,100,10
8,2023-08-30 00:00:00,0.9,100,10
9,2023-09-30 00:00:00,0.9,100,10


**Błędna częstotliwość spłat**

In [9]:
sched_payment_plan(freq_cde='m', 
                   payments_cnt=15, 
                   first_payment_dte='2022-12-30',  
                   proba_pct=0.9, 
                   capital_amt=100, 
                   interest_amt=10)

Niewłaściwa częstotliwość płatności: m


,data_platnosci,proba,kapital,odsetki


**Błędna data**

In [10]:
sched_payment_plan(freq_cde='Q', 
                   payments_cnt=3, 
                   first_payment_dte='2022-12-32',  
                   proba_pct=0.9, 
                   capital_amt=100, 
                   interest_amt=10)

Niewłaściwa data: 2022-12-32


,data_platnosci,proba,kapital,odsetki


**Błędne parametry planu - proba**

In [11]:
sched_payment_plan(freq_cde='M', 
                   payments_cnt=5, 
                   first_payment_dte='2022-12-30',  
                   proba_pct=-0.9, 
                   capital_amt=100, 
                   interest_amt=10)

Niewłaściwe parametry spłaty: -0.9 100 10


,data_platnosci,proba,kapital,odsetki


**Pływający dzień spłaty (29,30,31 - dzień miesiąca)**

In [12]:
sched_payment_plan(freq_cde='Y', 
                   payments_cnt=5, 
                   first_payment_dte='2024-02-29',  
                   proba_pct=0.9, 
                   capital_amt=100, 
                   interest_amt=10)

,data_platnosci,proba,kapital,odsetki
0,2024-02-29 00:00:00,0.9,100,10
1,2025-02-28 00:00:00,0.9,100,10
2,2026-02-28 00:00:00,0.9,100,10
3,2027-02-28 00:00:00,0.9,100,10
4,2028-02-29 00:00:00,0.9,100,10


### Przykład 2 - Harmonogram składany z wielu planów spłaty

[<Spis treści](#spis)

**Odesetki miesięcznie / Kapitał kwartalnie / Te same prawdopodobieństwa / różne dni spłat**

In [13]:
sp1=sched_payment_plan(freq_cde='M', 
                       payments_cnt=10, 
                       first_payment_dte='2022-12-30',  
                       proba_pct=0.9, 
                       capital_amt=0, 
                       interest_amt=10)

sp2=sched_payment_plan(freq_cde='Q', 
                       payments_cnt=3, 
                       first_payment_dte='2023-03-25',  
                       proba_pct=0.9, 
                       capital_amt=100, 
                       interest_amt=0)

sched_concat([sp1, sp2])


,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.9,0,10
1,2023-01-30,0.9,0,10
2,2023-02-28,0.9,0,10
3,2023-03-25,0.9,100,0
4,2023-03-30,0.9,0,10
5,2023-04-30,0.9,0,10
6,2023-05-30,0.9,0,10
7,2023-06-25,0.9,100,0
8,2023-06-30,0.9,0,10
9,2023-07-30,0.9,0,10


In [14]:
sp1=sched_payment_plan(freq_cde='M', 
                       payments_cnt=10, 
                       first_payment_dte='2022-12-30',  
                       proba_pct=0.9, 
                       capital_amt=0, 
                       interest_amt=10)

sp2=sched_payment_plan(freq_cde='M', 
                       payments_cnt=10, 
                       first_payment_dte='2023-03-25',  
                       proba_pct=0.9, capital_amt=100, 
                       interest_amt=0)

sched_concat([sp1, sp2])

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.9,0,10
1,2023-01-30,0.9,0,10
2,2023-02-28,0.9,0,10
3,2023-03-25,0.9,100,0
4,2023-03-30,0.9,0,10
5,2023-04-25,0.9,100,0
6,2023-04-30,0.9,0,10
7,2023-05-25,0.9,100,0
8,2023-05-30,0.9,0,10
9,2023-06-25,0.9,100,0


**Odesetki miesięcznie / Kapitał kwartalnie / Te same prawdopodobieństwa / Te same dni spłat**

In [15]:
sp1=sched_payment_plan(freq_cde='M', 
                       payments_cnt=10, 
                       first_payment_dte='2022-12-30',  
                       proba_pct=0.9, 
                       capital_amt=0, 
                       interest_amt=10)

sp2=sched_payment_plan(freq_cde='Q', 
                       payments_cnt=3, 
                       first_payment_dte='2023-03-30',  
                       proba_pct=0.9, 
                       capital_amt=100, 
                       interest_amt=0)

sched_concat([sp1, sp2])

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.9,0,10
1,2023-01-30,0.9,0,10
2,2023-02-28,0.9,0,10
3,2023-03-30,0.9,100,10
4,2023-04-30,0.9,0,10
5,2023-05-30,0.9,0,10
6,2023-06-30,0.9,100,10
7,2023-07-30,0.9,0,10
8,2023-08-30,0.9,0,10
9,2023-09-30,0.9,100,10


**Odesetki miesięcznie / Kapitał kwartalnie / różne prawdopodobieństwa / Te same dni spłat**

In [16]:
sp1=sched_payment_plan(freq_cde='M', 
                       payments_cnt=10, 
                       first_payment_dte='2022-12-30',  
                       proba_pct=0.23, 
                       capital_amt=0, 
                       interest_amt=10)

sp2=sched_payment_plan(freq_cde='Q', 
                       payments_cnt=3, 
                       first_payment_dte='2023-03-30',  
                       proba_pct=0.88, 
                       capital_amt=100, 
                       interest_amt=0)

sched_concat([sp1, sp2])

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.23,0,10
1,2023-01-30,0.23,0,10
2,2023-02-28,0.23,0,10
3,2023-03-30,0.23,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.23,0,10
6,2023-05-30,0.23,0,10
7,2023-06-30,0.23,0,10
8,2023-06-30,0.88,100,0
9,2023-07-30,0.23,0,10


### Przykład 3 - modyfikowanie płatności w zakresie dat

[<Spis treści](#spis)

**Harmonogram wyjściowy**

In [17]:
sp1=sched_payment_plan(freq_cde='M', 
                       payments_cnt=6, 
                       first_payment_dte='2022-12-30',  
                       proba_pct=0.11, 
                       capital_amt=0, 
                       interest_amt=10)

sp2=sched_payment_plan(freq_cde='Q', 
                       payments_cnt=3, 
                       first_payment_dte='2023-03-30',  
                       proba_pct=0.88, 
                       capital_amt=100, 
                       interest_amt=0)

sp0=sched_concat([sp1, sp2])
sp0

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


**Niewłaściwa data**

In [18]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2023-02-31', 
                end_dte='2023-07-15', 
                chng_proba_ind=0,
                chng_cap_ind=0,
                chng_int_ind=0, 
                new_proba=0,
                new_cap=0,
                new_int=0
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

Niewłaściwa data: 2023-02-31 2023-07-15


'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki


**Start_dte>end_dte**

In [19]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2023-10-31', 
                end_dte='2023-07-15', 
                chng_proba_ind=0,
                chng_cap_ind=0,
                chng_int_ind=0, 
                new_proba=0,
                new_cap=0,
                new_int=0
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

Start_dte późniejsze niż end_dte: 2023-10-31 00:00:00 2023-07-15 00:00:00


'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki


**Start_dte większe od daty końca harmonogramu**

In [20]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2024-03-31', 
                end_dte='2024-07-15', 
                chng_proba_ind=0,
                chng_cap_ind=0,
                chng_int_ind=1, 
                new_proba=0,
                new_cap=0,
                new_int=0
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

Start_dte późniejsze od max data raty


'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki


**End_dte mniejsze od daty początku harmonogramu**

In [21]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2021-03-31', 
                end_dte='2021-07-15', 
                chng_proba_ind=0,
                chng_cap_ind=0,
                chng_int_ind=1, 
                new_proba=0,
                new_cap=0,
                new_int=0
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

End_dte wcześniejsze niż min data raty


'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki


**Modyfikacja odsetek**

In [22]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                chng_int_ind=1,  
                new_int=33
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,33
6,2023-05-30,0.11,0,33
7,2023-06-30,0.88,100,33
8,2023-09-30,0.88,100,0


**Modyfikacja kapitału**

In [23]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                chng_cap_ind=1,  
                new_cap=222
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,222,10
6,2023-05-30,0.11,222,10
7,2023-06-30,0.88,222,0
8,2023-09-30,0.88,100,0


**Modyfikacja proba**

In [24]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                chng_proba_ind=1,  
                new_proba=0.44
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.44,0,10
6,2023-05-30,0.44,0,10
7,2023-06-30,0.44,100,0
8,2023-09-30,0.88,100,0


**Modyfikacja wszystki parametrów harmonogramu**

In [25]:
sp_start=sp0.copy()

sp=sched_modify(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                chng_proba_ind=1,
                chng_cap_ind=1,
                chng_int_ind=1, 
                new_proba=0.77,
                new_cap=777,
                new_int=77
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.77,777,77
6,2023-05-30,0.77,777,77
7,2023-06-30,0.77,777,77
8,2023-09-30,0.88,100,0


### Przykład 4 - przesuwanie płatności w harmonogramie

[<Spis treści](#spis)

**Harmonogram wyjściowy**

In [26]:
sp1=sched_payment_plan(freq_cde='M', 
                       payments_cnt=6, 
                       first_payment_dte='2022-12-30',  
                       proba_pct=0.11, 
                       capital_amt=0, 
                       interest_amt=10)

sp2=sched_payment_plan(freq_cde='Q', 
                       payments_cnt=3, 
                       first_payment_dte='2023-03-30',  
                       proba_pct=0.88, 
                       capital_amt=100, 
                       interest_amt=0)

sp0=sched_concat([sp1, sp2])
sp0

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


**Przesuwanie w przyszłość**

In [27]:
sp_start=sp0.copy()

sp=sched_shift(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                shift_months_cnt=20
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-09-30,0.88,100,0
6,2024-12-30,0.11,0,10
7,2025-01-30,0.11,0,10
8,2025-02-28,0.88,100,0


**Przyspieszanie płatności**

In [28]:
sp_start=sp0.copy()

sp=sched_shift(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                shift_months_cnt=-6
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-10-30,0.11,0,10
1,2022-11-30,0.11,0,10
2,2022-12-30,0.11,0,10
3,2022-12-30,0.88,100,0
4,2023-01-30,0.11,0,10
5,2023-02-28,0.11,0,10
6,2023-03-30,0.11,0,10
7,2023-03-30,0.88,100,0
8,2023-09-30,0.88,100,0


**Przesuwanie w ramach istniejącego zakresu - do przodu**

In [29]:
sp_start=sp0.copy()

sp=sched_shift(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                shift_months_cnt=3
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-07-30,0.11,0,10
6,2023-08-30,0.11,0,10
7,2023-09-30,0.88,200,0


**Przesuwanie w ramach istniejącego zakresu - do tyłu**

In [30]:
sp_start=sp0.copy()

sp=sched_shift(sched=sp_start, 
                start_dte='2023-03-31', 
                end_dte='2023-07-15', 
                shift_months_cnt=-2
               )

display('Harmonogram początkowy:',sp_start)
display('Harmonogram modyfikowany:',sp)

'Harmonogram początkowy:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


'Harmonogram modyfikowany:'

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,20
3,2023-03-30,0.11,0,20
4,2023-03-30,0.88,100,0
5,2023-04-30,0.88,100,0
6,2023-09-30,0.88,100,0


### Przykład 5 - usuwanie płatności sprzed daty analizy

[<Spis treści](#spis)

**Harmonogram startowy**

In [31]:
sp1=sched_payment_plan(freq_cde='M', 
                       payments_cnt=6, 
                       first_payment_dte='2022-12-30',  
                       proba_pct=0.11, 
                       capital_amt=0, 
                       interest_amt=10)

sp2=sched_payment_plan(freq_cde='Q', 
                       payments_cnt=3, 
                       first_payment_dte='2023-03-30',  
                       proba_pct=0.88, 
                       capital_amt=100, 
                       interest_amt=0)

sp0=sched_concat([sp1, sp2])
sp0

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


**Data danych w środku harmonogramu**

In [32]:
sched_dates_valid(sched=sp_start, 
                  data_dte='2023-05-31')

,data_platnosci,proba,kapital,odsetki
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


**Data danych sprzed harmonogramu**

In [33]:
sched_dates_valid(sched=sp_start, 
                  data_dte='2022-05-31')

,data_platnosci,proba,kapital,odsetki
0,2022-12-30,0.11,0,10
1,2023-01-30,0.11,0,10
2,2023-02-28,0.11,0,10
3,2023-03-30,0.11,0,10
4,2023-03-30,0.88,100,0
5,2023-04-30,0.11,0,10
6,2023-05-30,0.11,0,10
7,2023-06-30,0.88,100,0
8,2023-09-30,0.88,100,0


**Data danych późniejsza od dat z harmonogramu**

In [34]:
sched_dates_valid(sched=sp_start, 
                  data_dte='2025-05-31')

,data_platnosci,proba,kapital,odsetki
